# LFT Stage 8: 5-Qubit Code State |L₀⟩ Collapse Simulation
This notebook simulates the collapse of the 5-qubit quantum error correction code state |L₀⟩ under the Logic Field Theory (LFT) framework using logical strain minimization.

In [ ]:

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

# DAG definition
dag_L0 = {
    's_L0': [
        's_00000', 's_10010', 's_01001', 's_10100',
        's_01010', 's_11011', 's_00110', 's_11000',
        's_11101', 's_00011', 's_11110', 's_01111',
        's_10001', 's_00101', 's_01100', 's_10111'
    ]
}

# D values
D_L0 = {'s_L0': 2.0}
for s in dag_L0['s_L0']:
    D_L0[s] = 0.0


In [ ]:

def deterministic_collapse(dag, D_values, start):
    path = [start]
    current = start
    while current in dag:
        neighbors = dag[current]
        if not neighbors:
            break
        next_state = min(neighbors, key=lambda n: D_values[n])
        path.append(next_state)
        current = next_state
    return path

def stochastic_collapse(dag, D_values, start, beta):
    path = [start]
    current = start
    while current in dag:
        neighbors = dag[current]
        if not neighbors:
            break
        weights = [np.exp(-beta * D_values[n]) for n in neighbors]
        probs = [w / sum(weights) for w in weights]
        next_state = random.choices(neighbors, weights=probs)[0]
        path.append(next_state)
        current = next_state
    return path[-1]


In [ ]:

# Run stochastic simulations
def run_simulation(dag, D_values, start, beta, trials):
    final_counts = {}
    for _ in range(trials):
        final = stochastic_collapse(dag, D_values, start, beta)
        if final not in final_counts:
            final_counts[final] = 0
        final_counts[final] += 1
    total = sum(final_counts.values())
    return {k: v / total for k, v in final_counts.items()}

# Execute and display
result = run_simulation(dag_L0, D_L0, 's_L0', beta=2.0, trials=1000)
df = pd.DataFrame(result.items(), columns=["Outcome", "Probability"]).sort_values(by="Outcome")
display(df)

# Plotting
plt.figure(figsize=(12, 6))
plt.bar(df["Outcome"], df["Probability"])
plt.axhline(1/16, color='red', linestyle='--', label='Expected (1/16)')
plt.xticks(rotation=90)
plt.title("LFT Simulation: 5-Qubit Code State |L₀⟩ Collapse Outcome Probabilities")
plt.xlabel("Outcome State")
plt.ylabel("Probability")
plt.legend()
plt.tight_layout()
plt.show()
